<a href="https://colab.research.google.com/github/mwithgod3952/Machine_Learning_Theory_jh/blob/master/Autoencode%EB%A5%BC_%ED%86%B5%ED%95%9C_image_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import requsts

ModuleNotFoundError: ignored

In [46]:
!wget --header="Host: doc-10-1c-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-IN,en;q=0.9,en-GB;q=0.8,en-US;q=0.7,te;q=0.6,ru;q=0.5" --header="Referer: https://drive.google.com/" --header="Cookie: AUTH_4mcuou20mns0bnj0sduihld1f8t3235v_nonce=b9s3il0qbt66q; NID=204=Ays1_cpotq_cm0Nb8bI5w_F9WYRV9rPSgAxwJ4IQJPvPE5l1De1p5T0i_G8ks9GWBNkMcfcVizzpkqjbVmNmOMKYRPQ5VhQYLtQKRevszoS_v_6s2kqMVcPHQanN09BGmeFe45gU5QyrDCYmmtNMqeTsdqgq1PddAdhKBXWEJ-o; _ga=GA1.2.1000151777.1596382410" --header="Connection: keep-alive" "https://doc-10-1c-docs.googleusercontent.com/docs/securesc/snqegoftrruv9dh3ufo1tfoti4b5rfsr/lc95gejn67goa7fnhfbpt4tbq0jk9vn7/1597764600000/07496480791912752493/08286913039080874450/1VT-8w1rTT2GCE5IE5zFJPMzv7bqca-Ri?e=download&authuser=0&nonce=b9s3il0qbt66q&user=08286913039080874450&hash=lg9tmmkjjqqfmbdgtdr08daaltq6qpv4" -c -O 'dataset.zip'

--2021-08-21 14:51:05--  https://doc-10-1c-docs.googleusercontent.com/docs/securesc/snqegoftrruv9dh3ufo1tfoti4b5rfsr/lc95gejn67goa7fnhfbpt4tbq0jk9vn7/1597764600000/07496480791912752493/08286913039080874450/1VT-8w1rTT2GCE5IE5zFJPMzv7bqca-Ri?e=download&authuser=0&nonce=b9s3il0qbt66q&user=08286913039080874450&hash=lg9tmmkjjqqfmbdgtdr08daaltq6qpv4
Resolving doc-10-1c-docs.googleusercontent.com (doc-10-1c-docs.googleusercontent.com)... 173.194.210.132, 2607:f8b0:400c:c0f::84
Connecting to doc-10-1c-docs.googleusercontent.com (doc-10-1c-docs.googleusercontent.com)|173.194.210.132|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2021-08-21 14:51:05 ERROR 403: Forbidden.

